# Semantic Kernel ツール使用例

## 必要なパッケージのインポート

In [1]:
import json
import os

from dotenv import load_dotenv

from IPython.display import display, HTML

from typing import Annotated
from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## プラグインの作成

Semantic Kernelでは、エージェントが呼び出すツールとして「プラグイン」を使用します。プラグインは複数の`kernel_function`をグループとしてまとめることができます。

以下の例では、以下の2つの機能を持つ`DestinationsPlugin`を作成します：
1. `get_destinations`関数を使用して目的地のリストを提供
2. `get_availability`関数を使用して各目的地の空き状況を提供

In [ ]:
# サンプル用のプラグインを定義
class DestinationsPlugin:
    """バケーション先のリスト"""

    # @kernel_function デコレータは、このメソッドをSemantic Kernelが認識できるツールとして登録します
    # description パラメータは、AIエージェントがこのツールをいつ使用するべきかを理解するために重要です
    @kernel_function(description="バケーション先のリストを提供します。")
    def get_destinations(self) -> Annotated[str, "バケーション先を返します。"]:
        # 実際のアプリケーションでは、これはデータベースやAPIからデータを取得する場所になります
        return """
        バルセロナ、スペイン
        パリ、フランス
        ベルリン、ドイツ
        東京、日本
        ニューヨーク、アメリカ
        """

    # パラメータを受け取る関数の例 - AIエージェントは必要に応じて引数を提供します
    @kernel_function(description="目的地の空き状況を提供します。")
    def get_availability(
        self, destination: Annotated[str, "空き状況を確認する目的地。"]
    ) -> Annotated[str, "目的地の空き状況を返します。"]:
        # Annotated型ヒントは、Semantic Kernelがパラメータの目的を理解するのに役立ちます
        return """
        バルセロナ - 利用不可
        パリ - 利用可能
        ベルリン - 利用可能
        東京 - 利用不可
        ニューヨーク - 利用可能
        """

## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`ai_model_id`は`gpt-4o-mini`として定義されています。GitHub Modelsマーケットプレイスで利用可能な他のモデルに変更して、異なる結果を確認してみてください。

GitHub Modelsの`base_url`で使用される`Azure Inference SDK`を使用するため、Semantic Kernel内の`OpenAIChatCompletion`コネクタを使用します。他のモデルプロバイダーでSemantic Kernelを使用するための[利用可能なコネクタ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)も存在します。

In [ ]:
# 環境変数から設定を読み込み
load_dotenv()

# GitHub Models用のAsyncOpenAIクライアントを作成
# GitHub ModelsはOpenAI互換のAPIを提供するため、OpenAIクライアントを使用できます
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"),  # GitHubトークンを使用して認証
    base_url="https://models.inference.ai.azure.com/",  # GitHub ModelsのエンドポイントURL
)

# Semantic Kernel用のチャット完了サービスを作成
# このサービスがプラグインとLLMモデル間の橋渡し役として機能します
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",  # 使用するモデルを指定
    async_client=client,        # 上で作成したクライアントを渡す
)

## エージェントの作成

エージェント名と指示を設定してエージェントを作成します。

これらの設定を変更して、エージェントの応答の違いを確認できます。

In [ ]:
# ChatCompletionAgentを作成 - これがメインのAIエージェントです
agent = ChatCompletionAgent(
    service=chat_completion_service,              # 上で作成したチャット完了サービスを使用
    name="TravelAgent",                          # エージェントに名前を付ける（ログや識別に使用）
    instructions="旅行先とその空き状況についての質問に答えてください。",  # システムプロンプト - エージェントの役割を定義
    plugins=[DestinationsPlugin()],              # 利用可能なプラグインのリスト - エージェントが使用できるツール
)

## エージェントの実行

AIエージェントを実行します。このスニペットでは、`user_input`に2つのメッセージを追加して、エージェントがフォローアップの質問にどのように応答するかを示します。

エージェントは適切な関数を呼び出して、利用可能な目的地のリストを取得し、特定の場所の空き状況を確認する必要があります。

`user_inputs`を変更して、エージェントがどのように応答するかを確認できます。

In [ ]:
# テスト用のユーザー入力リスト - 異なるタイプの質問でエージェントの動作を確認
user_inputs = [
    "どんな目的地が利用可能ですか？",                    # リスト取得をテスト
    "バルセロナは利用可能ですか？",                      # 特定の都市の検索をテスト
    "ヨーロッパ以外で利用可能なバケーション先はありますか？",  # 複合的な質問をテスト
]

async def main():
    # スレッドはエージェントとの会話履歴を管理します
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        # ユーザー入力をHTMLで表示するための準備
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>ユーザー:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        # レスポンス処理用の変数を初期化
        agent_name = None
        full_response: list[str] = []      # エージェントの最終回答を格納
        function_calls: list[str] = []     # 関数呼び出しの詳細を格納

        # ストリーミング関数呼び出しを再構築するためのバッファ
        current_function_name = None
        argument_buffer = ""

        # エージェントをストリーミングモードで実行 - リアルタイムで応答を受信
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,  # 会話履歴を維持
        ):
            thread = response.thread        # スレッドを更新
            agent_name = response.name      # エージェント名を取得
            content_items = list(response.items)  # レスポンスの各項目を処理

            for item in content_items:
                # 関数呼び出しの開始を検出
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # 引数を蓄積（チャンクでストリーミング）
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                
                # 関数実行結果を検出
                elif isinstance(item, FunctionResultContent):
                    # 結果を表示する前に保留中の関数呼び出しを完了
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            # JSONとして整形を試行
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # 生の文字列として残す

                        function_calls.append(f"関数呼び出し: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\n関数の結果:\n\n{item.result}")
                
                # エージェントの通常のテキスト応答を検出
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # 関数呼び出しがあった場合、折りたたみ可能な詳細として表示
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>関数呼び出し（クリックして展開）</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        # エージェントの最終回答を表示
        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'アシスタント'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        # HTMLとして表示
        display(HTML(html_output))

# 非同期関数を実行
await main()

## 学習のポイント

### Semantic Kernel vs AutoGen の主な違い

- **Semantic Kernel**: プラグインとカーネル中心のアプローチ。`@kernel_function`デコレータを使用してクラスメソッドをツールとして簡単に登録できる
- **AutoGen**: エージェント中心のアプローチ。`FunctionTool`を明示的に作成してエージェントに登録する必要がある

### このサンプルから学べること

1. **プラグインの構造**: クラスベースでツールを整理し、複数の関連する機能をグループ化する手法
2. **`@kernel_function`デコレータ**: 既存のメソッドを簡単にAIツールに変換する方法
3. **ストリーミング応答**: リアルタイムでエージェントの応答と関数呼び出しを表示する手法
4. **スレッド管理**: 会話の履歴を管理し、コンテキストを維持する仕組み

### 次のステップ

- カスタムプラグインの開発とテスト
- 複数のプラグインを組み合わせた複雑なワークフロー
- プラグイン間でのデータ連携とパイプライン処理